In [23]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import glob
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [24]:
# Viena užduotis maždaug
# vienai paskaitai. Techniniai reikalavimai –
# Jupyter Notebook’as arba Streamlit aplinka. Grafikai turi turėti
# pavadinimus, ašys pavadintos prasmingai, jei grafike yra daugiau nei
# viena kreivė – legenda būtina. Panaudoti boxplot,
# swarmplot, heatmap, pairplot, (h)bar,
# histogram, scatter. Būtina pasirašyti 1 – 2
# sakinius į JupyterNotebook’ą apie tai, ką matote, ką galite
# pasakyti iš grafiko (ką matote grafike, 1-2 sakiniai vienam
# grafikui, tad jei turėsite 10 grafikų – 20 sakinių).

#  5        
      

# Duomenys:
# https://data.gov.lt/datasets/2829/
# - Stacionaro ir ambulatoriniai ligoniai, gydyti nuo traumų ir
# apsinuodijimų. Įvertinkite patiriamų traumų, apsinuodijimų
# dažnius/pasiskirstymą, sunkumą pagal amžių/amžiaus
# grupes, lytį, paros, metų, darbo dienų/švenčių/išeiginių. Ar
# traumų pobūdis/dažnis, apsinuodijimų kiekis, pobūdis, sunkumas
# priklauso nuo užfiksuotų meteorologinių sąlygų, mėnulio fazės,
# švenčių/laisvadienių?
# Duomenys:
# https://data.gov.lt/datasets/2793/#info
# – PAGD iškvietimai; https://data.gov.lt/datasets/1667/#info
# – Policijos
# iškvietimai. Ar
# iškvietimų dažnis/kiekis priklauso nuo paros,
# metų, darbo dienų/švenčių/išeiginių, mėnulio
# fazės, švenčių/laisvadienių?
# Mėnulio
# fazės turi būti pasiekiamos per http užklausas – iš esmės
# pasinaudojate web scrapingu, kad gautumėte norimą informaciją.
# Papildoma informacija -
# https://ligoniukasa.lrv.lt/lt/veiklos-sritys/informacija-gyventojams/gydymo-ir-sveikatos-prieziuros-paslaugos/stebejimo-paslaugos/
# ;

# TLK kodų paaiškinimai yra šiame excel faile:
# https://ligoniukasa.lrv.lt/uploads/ligoniukasa/documents/files/1(9).xlsx

# Jūs turėsite sujungti tris failus:
# https://data.gov.lt/datasets/2829/data/Pacientas/
# https://data.gov.lt/datasets/2829/data/Ligonine/
# ir pateiktą excel'io failą, kad turėtumėte vientisą lentelę su reikiamais duomenimis.
# Prie jų jungsite infromaciją iš
# https://astropixels.com/ephemeris/phasescat/phasescat.html
# ARBA day.lt - bus paprasčiau:
# https://day.lt/diena/2021.04.22    <--- adreso laukelyje įrašius norimą datą, gaunama informacija apie tą dieną.

# iš kurios, naudodami web scraping'ą, išgausite reikiamą informaciją apie fazes. (requests bibliotekos užteks)

In [25]:
# import requests
# r = requests.get('https://get.data.gov.lt/datasets/gov/hi/traumos_apsinuodijimai/Pacientas')
# ats = r.json()
# pandas_df = pd.DataFrame(data=ats["_data"])
# pandas_df.head()

In [26]:
df_iskvietimai = pd.read_json(r"C:\Users\pauli\Downloads\prir_2019_03_31.json")

failai=[]
for name in glob.glob(r'C:\Users\pauli\Downloads\iskvietimai\*.json'):
    failai.append(name)
    
for i in failai:
    df_iskvietimai = pd.concat([df_iskvietimai, pd.read_json(i)])

In [27]:
df_orai = pd.read_csv(r"C:\Users\pauli\Downloads\orai2014.csv")

failai=[]
for name in glob.glob(r'C:\Users\pauli\Downloads\orai\*.csv'):
    failai.append(name)
    
for i in failai:
    df_orai = pd.concat([df_orai, pd.read_csv(i)])

In [37]:
df_orai.head(1)

,obs_time_utc,air_temperature,feels_like_temperature,wind_speed,wind_gust,wind_direction,cloud_cover,sea_level_pressure,relative_humidity,precipitation,condition_code
0,2014-01-01 00:00:00,-1.4,-5.1,2.9,NaN,175.0,88.0,1027.9,89.0,0.0,cloudy


In [28]:
df_pacientas = pd.read_csv(r"C:\Users\pauli\Downloads\Pacientas.csv", low_memory=False)

In [35]:
df_pacientas.head(1)

,_type,_id,_revision,_page.next,vda_pk,paciento_id,atvejo_id,gyd_lap_pradzia,lytis,vda_gim_metai,savivaldybe,gyvenviete,gydymas,stacionaro_tipas,tlk_trauma,tlk_aplinkybe,tlk_veikla,tlk_vieta
0,datasets/gov/hi/traumos_apsinuodijimai/Pacientas,fe5742d1-21bb-4897-9716-0ea67e22e005,f68b114b-36e7-4d6e-a072-919c64f98594,NaN,000002a37f16bf4,24cdba64a645d60,062bca343d25de9,2013-01-01,Vyras,1998-01-01,Kretingos r. sav.,Nežinoma,Ambulatorinis,NaN,S42.43,NaN,NaN,NaN


In [29]:
df_tlk = pd.read_excel(r"C:\Users\pauli\Downloads\1(9).xlsx")

In [36]:
df_tlk.head(1)

,TLK-10-AM Kodas,TLK-10-AM Kodo pavadinimas,Kodo lygis,Žvaigždutė,Apskritimas,Kryželis,Poskyrio eil. nr.,Poskyrio blokas,Poskyrio pavadinimas,Skyriaus nr.,Skyriaus blokas,Skyriaus pavadinimas,Data nuo,Data iki
0,A00,Cholera,1,0,0,0,1,(A00-A09),ŽARNYNO INFEKCINĖS LIGOS,1,(A00-B99),TAM TIKROS INFEKCINĖS IR PARAZITŲ SUKELIAMOS L...,1997-01-01,2100-01-01


In [30]:
df_ivykis = pd.read_csv(r"C:\Users\pauli\Downloads\Ivykis.csv", low_memory=False)

In [31]:
df_menulis = pd.read_csv(r"C:\Users\pauli\Desktop\DataScienceNotebooks\Studentai\Paulius Vanagas\menulis.csv", low_memory=False)

In [32]:
# start_date = datetime(2003, 1, 1)
# end_date = datetime(2024, 12, 30)

# date_list = [(start_date + timedelta(days=i)).strftime("%Y-%m-%d") 
#              for i in range((end_date - start_date).days + 1)]

# for i in range(0, len(date_list)):
#     date_list[i] = date_list[i].replace('-', '.')

In [33]:
# url = 'https://day.lt/diena/'
# menulis=[]

# for i in range(0, len(date_list)):
#     try:
#         page = requests.get(url+date_list[i])
#         source = page.content
#         bs = BeautifulSoup(source, 'html.parser')
        
#         a = bs.find('div', {'class':'sun-moon-container m-4'})
#         a = a.find('a', {'title':'Apie dabartinį mėnulį'})
#         a = a.find('img')
        
#         menulis.append(a['alt'])
        
#     except:
#         menulis.append(menulis[-1])

In [34]:
# for i in range(0, len(menulis)):
#     menulis[i] = menulis[i].split(' ')[0]

# for i in range(0, len(date_list)):
#     date_list[i] = date_list[i].replace('.', '-')

# df_menulis = pd.DataFrame({"data": date_list, "menulis":menulis })
# df_menulis.to_csv("menulis.csv", index=False)

# df_menulis.head()